In [1]:
import pandas as pd
import numpy as np
import requests
df = pd.read_csv('北京商铺数据.csv',error_bad_lines=False)

In [2]:
df.head()

,zhuanrangfei,rental_price,pay_method,jianzhumianji,shiyongmianji,shengyuzuqi,zhuanrangfangshi,floor,zhuangxiu,building_type,...,loupanmingcheng,loupandizhi,miankuan,jinshen,cenggao,elvator_number,service_price,function_list,release_date,url
0,NaN,37500元/月,押一付三,120㎡,NaN,NaN,NaN,第1层(共1层),精装修,住宅底商,...,通州区云景东路时尚街区,通州区云景东路8号,4.00米,30.00米,5.00米,NaN,NaN,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",2019/8/20 16:49:56,https://shop.fang.com/zu/3_427451616.html
1,NaN,134400元/月,押二付三,392265.00㎡,NaN,NaN,NaN,第1层(共1层),精装修,写字楼配套底商,...,望京SOHO商业,望京街与阜安西路交叉路口（望京街中部）,15.00米,15.00米,4.50米,客梯4个，货梯2个,租金已包含,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",2019/9/10 15:15:53,https://shop.fang.com/zu/3_438557080.html
2,NaN,50000元/月,押一付三,200㎡,NaN,NaN,NaN,第1层(共6层),简装修,住宅底商,...,建外SOHO商业,东三环中路39号,5.00米,4.00米,3.00米,NaN,租金已包含,"网络,暖气,停车位,客梯,货梯,燃气",2019/9/10 10:02:53,https://shop.fang.com/zu/3_438541177.html
3,NaN,150000元/月,押二付三,330000.00㎡,NaN,NaN,NaN,第1层(共1层),精装修,临街门面,...,银河SOHO商铺,小牌坊胡同甲七号,35.00米,20.00米,5.20米,客梯18个，货梯6个,33.00元/㎡·月,"网络,暖气,停车位,中央空调,上水,下水,排烟",2019/7/20 14:03:03,https://shop.fang.com/zu/3_436067569.html
4,NaN,24000元/月,押三付三,100000.00㎡,NaN,NaN,NaN,第1层(共30层),精装修,写字楼配套底商,...,建外SOHO商铺,东三环中路39号,8.00米,8.00米,3.00米,客梯12个，货梯2个,租金已包含,"中央空调,上水,下水,可明火,排烟,排污",2019/8/8 17:49:04,https://shop.fang.com/zu/3_437076463.html


In [3]:
lst_lpname = df['loupanmingcheng'].value_counts().index.tolist()

lst_lploc = df['loupandizhi'].value_counts().index.tolist()

In [4]:
city = '北京'
dict_location = {}
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in range(len(lst_lpname)):
    location = lst_lpname[i]
    url = url_pre.format(My_key, location, city)
    res = requests.get(url).json()
    if res['status'] == '0':
        location = lst_lploc[i]
        url = url_pre.format(My_key, location, city)
        res = requests.get(url).json()
        if res['status'] == '0':
            dict_location[lst_lpname[i]] = np.nan
            counter +=1
        else:
            try: 
                dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                if counter % 250 == 0:
                    print('one batch(250) processed')
            except:
                dict_location[lst_lpname[i]] = np.nan
            
    else:
        try: 
            dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            location = lst_lploc[i]
            url = url_pre.format(My_key, location, city)
            res = requests.get(url).json()
            if res['status'] == '0':
                dict_location[lst_lpname[i]] = np.nan
                counter +=1
            else:
                try: 
                    dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                    if counter % 250 == 0:
                        print('one batch(250) processed')
                except:
                    dict_location[lst_lpname[i]] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed


In [10]:
df['location_coords'] = np.nan

counter = 0
for i in lst_lpname:
    counter += 1
    df_temp = df[df['loupanmingcheng'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,22] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [11]:
df.head()

,zhuanrangfei,rental_price,pay_method,jianzhumianji,shiyongmianji,shengyuzuqi,zhuanrangfangshi,floor,zhuangxiu,building_type,...,loupandizhi,miankuan,jinshen,cenggao,elvator_number,service_price,function_list,release_date,url,location_coords
0,NaN,37500元/月,押一付三,120㎡,NaN,NaN,NaN,第1层(共1层),精装修,住宅底商,...,通州区云景东路8号,4.00米,30.00米,5.00米,NaN,NaN,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",2019/8/20 16:49:56,https://shop.fang.com/zu/3_427451616.html,"116.667013,39.882850"
1,NaN,134400元/月,押二付三,392265.00㎡,NaN,NaN,NaN,第1层(共1层),精装修,写字楼配套底商,...,望京街与阜安西路交叉路口（望京街中部）,15.00米,15.00米,4.50米,客梯4个，货梯2个,租金已包含,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",2019/9/10 15:15:53,https://shop.fang.com/zu/3_438557080.html,"116.481050,39.996794"
2,NaN,50000元/月,押一付三,200㎡,NaN,NaN,NaN,第1层(共6层),简装修,住宅底商,...,东三环中路39号,5.00米,4.00米,3.00米,NaN,租金已包含,"网络,暖气,停车位,客梯,货梯,燃气",2019/9/10 10:02:53,https://shop.fang.com/zu/3_438541177.html,"116.458705,39.906458"
3,NaN,150000元/月,押二付三,330000.00㎡,NaN,NaN,NaN,第1层(共1层),精装修,临街门面,...,小牌坊胡同甲七号,35.00米,20.00米,5.20米,客梯18个，货梯6个,33.00元/㎡·月,"网络,暖气,停车位,中央空调,上水,下水,排烟",2019/7/20 14:03:03,https://shop.fang.com/zu/3_436067569.html,"116.433366,39.920552"
4,NaN,24000元/月,押三付三,100000.00㎡,NaN,NaN,NaN,第1层(共30层),精装修,写字楼配套底商,...,东三环中路39号,8.00米,8.00米,3.00米,客梯12个，货梯2个,租金已包含,"中央空调,上水,下水,可明火,排烟,排污",2019/8/8 17:49:04,https://shop.fang.com/zu/3_437076463.html,"116.458705,39.906458"


In [12]:
len(df)

1606

In [13]:
df.to_csv('北京商铺地址数据.csv', encoding = 'utf-8-sig',index = False)